In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np 
import glob 
import warnings 
from scipy.stats import norm
from IPython.display import display, HTML
import scipy
import os
warnings.filterwarnings("ignore")
%matplotlib inline
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
sns.set()

In [14]:
leagues = ['EPL','La_Liga','Bundesliga','Serie_A','Ligue_1','RFPL']

path = r'C:\Users\user\Desktop\Kaggle\Football stats analysis project\Data\UnderStat'

In [4]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [17]:
def getLeagueTeams(league):    
    dfOut =[]
    values=[]
    #Establish Connection
    driver.get('https://understat.com/league/'+league)
    #Get the relevant object
    base = driver.find_element_by_id('league-chemp')
    #Get Rows
    rows = base.find_elements_by_tag_name('tr')
    for item in range(1,len(rows)):
        #Get elements from the row
        row = rows[item].find_elements_by_tag_name('td')
        #Add those elements to the values list
        values.append([row[i].text for i in range(1,len(row))])
    dfOut = pd.DataFrame(values,columns=['team','matches','wins','draws','losses',
                                             'goals_for','goals_against','points','xG','xGA','xPTS'])
    return dfOut
    
def getLeagueStats(leagues):
    #Go through every league
    for league in leagues:
        df = getLeagueTeams(league)
        df['League'] = league
        df.to_csv(path + '\Team_Stats_%s'%league + '.csv')
    return print('Every league team stats crawled!')

getLeagueStats(leagues)    

Every league team stats crawled!


In [21]:
os.chdir(r'C:\Users\user\Desktop\Kaggle\Football stats analysis project\Data\UnderStat')
def loadData(path):
    allFiles = glob.glob(path)
    dfOut = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=0, header=0)
        list_.append(df)
    dfOut = pd.concat(list_)
    return dfOut
df = loadData('Team_Stats_*.csv')
team_names = df['team'].values
team_names = [i.replace(' ','_') for i in team_names]

In [40]:
"""Function mines players stats Data"""
def getPlayerStats(name):
    values = []
    driver.get('https://understat.com/team/'+name+'/2018')
    baseTable = driver.find_element_by_id('team-players')
    tableRows = baseTable.find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values,columns=['Name','Position','Apps','Min','Goals_Scored','Assists','Sh90',
                                     'KP90','xG','xA','xG90','xA90'])
    return df 


def getTeamPlayersStats(team_names):
    for name in team_names:
        df = getPlayerStats(name)
        df['Team'] = name
        df.to_csv(path + '\Player_Stats_%s'%name + '.csv')
    return print('Done')

getTeamPlayersStats(team_names)

Done


In [43]:
"""summary"""
name = 'Manchester_United'

values = []
driver.get('https://understat.com/team/'+name+'/2018')
baseTable = driver.find_element_by_id('team-statistics')
tableRows = baseTable.find_elements_by_tag_name('tr')
for item in range(1,len(tableRows)):
    row = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(1,len(row))])
df = pd.DataFrame(values)

In [68]:
"""formation"""

name = 'Manchester_United'

values = []
driver.get('https://understat.com/team/'+name+'/2018')
driver.find_element_by_xpath("//label[@for='statistic-3']").click()
baseTable = driver.find_element_by_id('team-statistics')
tableRows = baseTable.find_elements_by_tag_name('tr')
for item in range(1,len(tableRows)):
    row = tableRows[item].find_elements_by_tag_name('td')
    values.append([row[i].text for i in range(1,len(row))])
df = pd.DataFrame(values)

In [69]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,Goal diff 0,1284,154,20,170,15,20.43+0.43,17.54+2.54,2.89,1.43,1.23
1,Goal diff +1,556,82,14,86,3,9.16-4.84,6.49+3.49,2.67,1.48,1.05
2,Goal diff > +1,422,76,8,66,9,13.24+5.24,9.96+0.96,3.27,2.82,2.12
3,Goal diff -1,318,56,9,49,12,8.00-1.00,7.38-4.62,0.63,2.26,2.09
4,Goal diff < -1,253,43,7,21,1,6.49-0.51,2.93+1.93,3.56,2.31,1.04
